In [ ]:
import glob
import re
import tarfile

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

### How to use this script

After the completion of a series of simulations, Restart and Run All on this script after changing the following:
* Make sure `dirnames` and `scenario_names` align with with names in your `config.yml` file.
* If you'd like to plot at a different death rate, change the name (defaults are 'low', 'med', and 'high'), but depends on your `config.yml`
* Point `tfile = tarfile.open(...)` to the file your simulation produced

At the bottom of this file, you'll find a couple plots, one of hospital bed and ICU bed occupancy by day and some charts of various important statistics in detail by period. You can copy these wherever you'd like them.

In [ ]:
dirnames = ['Mid', 'Mild', 'Severe', 'NoCoronavirus', 'MMTestIsolate', 'MMInfluenza']
scenario_names = ['1918-Style', 'Reopen', 'Test and Isolate', 'Did nothing', '(MM) Test and Isolate', '(MM) 1918-Style']
death_rate_name = 'med'

In [ ]:
tfile = tarfile.open('output/output-2020-04-07T23-49-49.tar.gz')

In [ ]:
dfs = []
for member in tfile.getmembers():
    match = re.match(f'hospitalization/model_output/minimal_(.*)/{death_rate_name}.*csv', member.name)
    if match:
        dirname = match.groups()[0]
        scenario_name = scenario_names[dirnames.index(dirname)]
        f = tfile.extractfile(member)
        df = pd.read_csv(f)
        df = df[df['geoid'] // 1000 == 44]  # Pull out RI from the simulation
        df = df[df['comp'] == 'diffI']
        df['scenario_name'] = scenario_name
        dfs.append(df)

In [ ]:
df = pd.concat(dfs)

In [ ]:
# There are 5 counties in RI
assert (df.groupby(['scenario_name', 'time', 'sim_num']).size() == 5).all()

In [ ]:
# Add all counties in RI
grouped = df.groupby(['sim_num', 'time', 'scenario_name'])[['hosp_curr', 'icu_curr', 'incidD', 'incidH', 'incidI']].sum().reset_index()
grouped['time'] = pd.to_datetime(grouped.time)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

grouped['Scenario'] = grouped.scenario_name
blah = sns.lineplot(
    data = grouped,
    x = 'time',
    y = 'hosp_curr',
    hue='Scenario',
    ax = ax
)

plt.xticks(rotation=45)
plt.ylabel('Number Hospital Beds Occupied per Day')

box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

# Put a legend to the right of the current axis
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.grid(axis='both')
plt.title('Median bolded; 95% CI in bands')

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

grouped['Scenario'] = grouped.scenario_name
blah = sns.lineplot(
    data = grouped,
    x = 'time',
    y = 'icu_curr',
    hue='Scenario',
    ax = ax
)

plt.xticks(rotation=45)
plt.ylabel('Number ICU Beds Occupied per Day')

box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

# Put a legend to the right of the current axis
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.grid(axis='both')
plt.title('Median bolded; 95% CI in bands')

In [ ]:
def low_quantile(x): return np.quantile(x, 0.025)
def high_quantile(x): return np.quantile(x, 0.975)
def create_pivot_display(focus, col):
    pivoted = focus.groupby(['scenario_name', 'months', 'sim_num'])[col].sum().reset_index()\
        .pivot_table(index='scenario_name', columns='months', values=col, aggfunc=[
        'median', low_quantile, high_quantile]).astype(int)
    return (pivoted['median'].applymap(lambda x: '{:,}'.format(x)) + 
            '  (' + pivoted['low_quantile'].applymap(lambda x: '{:,}'.format(x)) +
            ' – ' + pivoted['high_quantile'].applymap(lambda x: '{:,}'.format(x)) +
            ')')

In [ ]:
focus = grouped[grouped['time'] >= '03-01-2020'].copy()
focus['period'] = focus['time'].dt.month // 2
focus['months'] = focus.period.map({1: '03-04', 2: '05-06', 3: '07-08', 4: '09-10'})

In [ ]:
print('Peak hospital occupancy in period (median + 95% CI)')
create_pivot_display(focus, 'hosp_curr')

In [ ]:
print('Deaths in period (median + 95% CI)')
create_pivot_display(focus, 'incidD')

In [ ]:
print('Hospital admissions in period (median + 95% CI)')
create_pivot_display(focus, 'incidH')

In [ ]:
print('Infections in period (median + 95% CI)')
create_pivot_display(focus, 'incidI')

In [ ]:
### TODO(khw): Further visualizations pivoting on IFR